In [ ]:
print()

In [ ]:
import pandas as pd
import numpy as np

# 12 features + 1 class
cols = ['number','snrtotal','snrave','numtotal','elesnrtotal','elesnrave','snr20num','snr15num','gpsz_tmean','gpsz_stdz','tenTotal','R','Class']
cols = ['snrtotal','snrave','elesnrtotal','elesnrave','gpsz_tmean','gpsz_stdz','tenTotal','R','Class']
cols_update = ['snrave','numtotal','elesnrtotal','elesnrave','snr15num','gpsz_tmean', 'tenTotal','R','Class']
cols_update = ['number','snrtotal','snrave','numtotal','elesnrtotal','elesnrave','snr20num','snr15num','gpsz_tmean','tenTotal','R','Class']
cols_update = cols

# collected data
dataxy_sub01 = pd.read_csv("./test01.csv") # "地铁室外跑"
dataxy_sub02 = pd.read_csv("./test02.csv") # 地铁站大棚
dataxy_sub03 = pd.read_csv("./test03.csv") # 楼梯电梯地下地铁站口
dataxy_sub04 = pd.read_csv("./test04.csv") # 地铁场景的数据
dataxy_sub05 = pd.read_csv("./test05.csv") # "主流场景"
print(dataxy_sub01.shape, dataxy_sub02.shape, dataxy_sub03.shape, dataxy_sub04.shape, dataxy_sub05.shape)

dataxy_sub00 = pd.concat([dataxy_sub02, dataxy_sub03, dataxy_sub04, dataxy_sub05])
print(dataxy_sub00.shape)

# log data
log_dataxy01 = pd.read_csv("./log_data01.csv") 
log_dataxy02 = pd.read_csv("./log_data02.csv") 
log_dataxy03 = pd.read_csv("./log_data03.csv") 
log_dataxy04 = pd.read_csv("./log_data04.csv") 
print(log_dataxy01.shape, log_dataxy02.shape, log_dataxy03.shape, log_dataxy04.shape)

(963, 13) (1115, 13) (1578, 13) (4982, 13) (21208, 13)
(28883, 13)
(19237, 13) (12586, 13) (26422, 13) (21264, 13)


In [ ]:
#
import warnings
warnings.filterwarnings("ignore")

from sklearn.utils import shuffle

def shuffle_split(df_tmp, cols, ratio=0.25):
  df_shuffled = df_tmp.copy()
  df_shuffled = shuffle(df_shuffled, random_state=0)
  # set "indoor" as 1, "outdoor" as 0
  df_shuffled["Class"] = df_shuffled["Class"].apply(lambda x: 1 if x=="indoor" else 0)
  # split as "training" & "testing"
  L = len(df_tmp)
  n1 = int(L*ratio)
  train_xy = df_shuffled[:n1]
  test_xy = df_shuffled[n1:]
  # features + class/label
  X_train = train_xy[cols[:-1]]
  y_train = train_xy[cols[-1]]
  X_test = test_xy[cols[:-1]]
  y_test = test_xy[cols[-1]]
  return X_train, y_train, X_test, y_test

def just_split(df_tmp, cols):
  df_shuffled = shuffle(df_tmp, random_state=0)
  df_shuffled["Class"] = df_shuffled["Class"].apply(lambda x: 1 if x=="indoor" else 0)
  X = df_shuffled[cols[:-1]]
  y = df_shuffled[cols[-1]]
  return X, y

def log_evaluation(y_test, y_pred, verbose=True):
  yt = {0:0, 1:0}
  yp = {0:0, 1:0}
  iv = 0
  ov = 0
  #
  i = 0
  L = len(y_test)
  while i<L:
    vt = y_test[i]
    vp = y_pred[i]
    #
    if vt == 0:
      yt[0] = yt[0] + 1
    else:
      yt[1] = yt[1] + 1
    if vp == 0:
      yp[0] = yp[0] + 1
    else:
      yp[1] = yp[1] + 1
    #
    if vt==vp:
      if vt==0:
        ov = ov + 1
      else:
        iv = iv + 1
    #
    i = i + 1
  #
  yt0 = round(yt[0]/L, 3)
  yt1 = round(yt[1]/L, 3)
  yp0 = round(yp[0]/L, 3)
  yp1 = round(yp[1]/L, 3)
  if verbose:
    print("prediction by: \t\t wifi\t\t model:")
    print("outdoor - 0: \t\t", yt0, yt[0], "\t", yp0, yp[0])
    print("indoor - 1: \t\t", yt1, yt[1], "\t", yp1, yp[1])
    print("comparison between wifi & model: ")
    print("outdoor overlapping - 0: \t", round(ov/L, 3), ov)
    print("indoor overlapping - 1: \t", round(iv/L, 3), iv)
  return yt, yp, iv, ov, L

In [ ]:
# NN
X_train, y_train, X_test, y_test = shuffle_split(dataxy_sub00, cols, ratio=0.7)
X_log, y_log = just_split(log_dataxy01, cols)
y_log

15458    0
15175    0
9132     0
10435    0
3249     0
        ..
9225     0
13123    0
9845     0
10799    0
2732     0
Name: Class, Length: 19237, dtype: int64

In [ ]:
# NN
X_train, y_train, X_test, y_test = shuffle_split(dataxy_sub00, cols, ratio=0.7)
X_log, y_log = just_split(log_dataxy01, cols)

import tensorflow
print(tensorflow.__version__)

from tensorflow.keras import Model
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
from sklearn import preprocessing


"""from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)"""

def NN_model(X_train, y_train, X_test, y_test, X_log, y_log, count, n1=10, n2=6, n3=3):
  n_features = 8 # 12
  model = Sequential()
  model.add(Dense(n1, activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)))
  model.add(Dense(n2, activation='relu'))
  model.add(Dense(n3, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  # compile the model
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  # fit the model
  model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)
  loss, acc = model.evaluate(X_test, y_test, verbose=0)
  print("loss: ", loss, "\t accuracy for testing: ", acc)
  #
  for ratio in range(3, 8):
    y_pred = (model.predict(X_log) > ratio/10).astype("int32")
    y_pred = y_pred.reshape(len(y_pred),)
    yt, yp, iv, ov, L = log_evaluation(y_log, y_pred, False)
    #
    print("--------------------------------------- ", iv/yt[1], ov/yt[0], "\t", n1, n2, n3, ratio)
    if iv/yt[1] > 0.6 and ov/yt[0] > 0.7:
      model.save("my_model_0825_new_"+str(count)+".h5")
  return yt, yp, iv, ov, L

search_flag = False
search_flag = True
count = 0
if search_flag:
  for n1 in range(10, 11):
    for n2 in range(6, 10):
      for n3 in range(3, 10):
        tmp = NN_model(X_train, y_train, X_test, y_test, X_log, y_log, count, n1, n2, n3)
        count = count + 1
# 10, 3, 4-9

2.6.0
loss:  0.20440861582756042 	 accuracy for testing:  0.9227928519248962
---------------------------------------  0.32792207792207795 0.6784173903042153 	 10 6 3 3
---------------------------------------  0.25703463203463206 0.7398355282074875 	 10 6 3 4
---------------------------------------  0.23593073593073594 0.7705445971591236 	 10 6 3 5
---------------------------------------  0.21103896103896103 0.7910748174133073 	 10 6 3 6
---------------------------------------  0.1937229437229437 0.8094197481166254 	 10 6 3 7
loss:  0.2960417568683624 	 accuracy for testing:  0.8724755048751831
---------------------------------------  0.12012987012987013 0.8715854850767727 	 10 6 4 3
---------------------------------------  0.11038961038961038 0.8817643337742251 	 10 6 4 4
---------------------------------------  0.10227272727272728 0.8906205072172063 	 10 6 4 5
---------------------------------------  0.09686147186147186 0.898499051124274 	 10 6 4 6
------------------------------------